In [ ]:
import ibis
from ibis import _

ibis.options.interactive = True
import pandas as pd
pd.set_option('max_colwidth', 400)

In [ ]:
con = ibis.datasette.connect("https://scotrail.datasette.io/scotrail")

In [ ]:
con.list_tables()

In [ ]:
t = con.tables.announcements

In [ ]:
t.head()

In [ ]:
t.select("Category").distinct()

In [ ]:
t.filter((_.Category == "Reason") & _.Transcription.contains("theft")).select("Transcription")

In [ ]:
def random(category):
    return (
        con.tables.announcements
        .filter(_.Category == category)
        .sort_by(ibis.random())
        .select("Transcription", "mp3")
        .limit(1)
    )

def phrase(t):
    return (
        con.tables.announcements
        .filter(_.Transcription == t)
        .select("Transcription", "mp3")
        .limit(1)
    )

query = ibis.union(
    random("Apology"),
    random("Train operating company"),
    random("Destination"),
    phrase("has been cancelled"),
    phrase("due to"),
    random("Reason"),
)

In [ ]:
query.execute()

In [ ]:
query.Transcription.group_concat(" ")

In [ ]:
import tempfile
import os
import pydub
import httpx
import ipywidgets
from IPython.display import Audio, display

output = ipywidgets.Output()
button = ipywidgets.Button(description='Generate Apology', icon="repeat")       
UI = ipywidgets.VBox([button, output])


def concatenate_mp3s(urls: list[str]) -> bytes:
    with httpx.Client(follow_redirects=True) as client, tempfile.TemporaryDirectory() as tempdir:
        output = None
        for i, url in enumerate(urls):
            path = os.path.join(tempdir, f"part{i}.mp3")
            with open(path, "wb") as f:
                resp = client.get(url)
                resp.raise_for_status()
                f.write(resp.content)
            part = pydub.AudioSegment.from_mp3(path)
            if output is None:
                output = part
            else:
                output = output + part
        out_path = os.path.join(tempdir, "output.mp3")
        output.export(out_path, format="mp3")
        with open(out_path, "rb") as f:
            return f.read()


@button.on_click
def on_click(*args):
    output.clear_output()
    result = query.execute()
    msg = " ".join(result.Transcription)
    mp3 = concatenate_mp3s(result.mp3)
    with output:
        print(msg)
        display(Audio(mp3))

        
UI